# Descomposición en Valores Singulares (SVD)
## Implementación basada en las diapositivas del curso

En este notebook se implementa:
 1. Descomposición SVD de una matriz
 2. Aproximación de rango bajo
 3. Pseudoinversa de Moore-Penrose
 4. Resolución de problemas de mínimos cuadrados

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Configuración para mejor visualización
np.set_printoptions(precision=4, suppress=True)
plt.style.use('seaborn-v0_8-darkgrid')

### 1. Descomposición SVD Básica
 
Una matriz $A$ se puede escribir como suma de $r$ matrices de rango uno:

$$A = \sigma_1 u_1 v_1^t + \sigma_2 u_2 v_2^t + \ldots + \sigma_r u_r v_r^t$$

Donde $u_1, u_2, \ldots, u_r$ y $v_1, v_2, \ldots, v_r$ son las primeras columnas de $U$ y $V$.


En esta sección implementamos una descomposición SVD siguiendo la siguiente estructura:

1. Reducción de la matriz A a forma bidiagonal mediante transformaciones ortogonales.
2. Cálculo de la SVD de la matriz bidiagonal obtenida.
3. Composición final de los factores para obtener la SVD completa de A.

In [2]:
import numpy as np

def householder_vector(x):
    '''Calcula el vector de Householder para un vector dado x.'''
    e1 = np.zeros_like(x)
    e1[0] = 1.0
    alpha = np.linalg.norm(x)
    if alpha == 0:
        return e1
    v = x + np.sign(x[0]) * alpha * e1
    return v / np.linalg.norm(v)


#### Paso 1: Reducción de A a forma bidiagonal B

Usamos reflexiones de Householder para anular:

- todos los elementos por debajo de la diagonal (Householder por columnas),
- todos los elementos a la derecha de la diagonal (Householder por filas).


Obtenemos matrices ortogonales \( U_1 \) y \( V_1 \) tales que

\[
A = U_1 \, B \, V_1^T,
\]

donde \( B \) es bidiagonal.

In [9]:
def bidiagonal_reduction(A):
    '''Reduce una matriz A a forma bidiagonal usando transformaciones de Householder.'''
    
    A = A.copy().astype(float)
    m, n = A.shape
    U = np.eye(m)
    V = np.eye(n)

    for k in range(min(m, n)):

        # Columas
        x = A[k:, k]
        v = householder_vector(x)
        H = np.eye(m)
        H[k:, k:] -= 2 * np.outer(v, v)

        A = H @ A
        U = U @ H

        # Filas
        if k < n - 1:
            x = A[k, k+1:]
            v = householder_vector(x)
            G = np.eye(n)
            G[k+1:, k+1:] -= 2 * np.outer(v, v)

            A = A @ G
            V = V @ G

    return U, A, V  # A ahora es bidiagonal


#### Paso 2: Cálculo de la SVD de la matriz bidiagonal B

1. Construimos la matriz simétrica \( B^T B \).
2. Obtenemos autovalores y autovectores (equivalente a obtener \( V_2 \)).
3. Calculamos los valores singulares como  
   $$ \sigma_i = \sqrt{\lambda_i}. $$
4. Obtenemos \( U_2 \) a partir de  
   $$ u_i = \frac{1}{\sigma_i}\, B\, v_i. $$

In [23]:
def svd_bidiagonal(B):
    '''Calcula la SVD de una matriz bidiagonal B.'''

    # Construimos la matriz simétrica
    BtB = B.T @ B

    # Autovalores y autovectores
    eigvals, V2 = np.linalg.eigh(BtB)

    # Orden descendente
    idx = np.argsort(eigvals)[::-1]
    eigvals = eigvals[idx]
    V2 = V2[:, idx]

    # Valores singulares
    s = np.sqrt(np.maximum(eigvals, 0))

    # Calculamos U2 usando u_i = (1 / sigma_i) * B * v_i
    m, n = B.shape
    U2 = np.zeros((m, len(s)))

    for i in range(len(s)):
        if s[i] > 1e-12:
            u = (B @ V2[:, i]) / s[i]
        else:
            u = np.zeros(m)

        # Gram–Schmidt robusto
        for j in range(i):
            proj = np.dot(U2[:, j], u)
            u = u - proj * U2[:, j]

        norm_u = np.linalg.norm(u)
        if norm_u > 1e-12:
            U2[:, i] = u / norm_u
        else:
            U2[:, i] = np.zeros(m)

    # --- Alinear signos ---
    for i in range(len(s)):
        if np.abs(U2[:, i]).max() > 0:
            if U2[np.argmax(np.abs(U2[:, i])), i] < 0:
                U2[:, i] *= -1
                V2[:, i] *= -1

    return U2, s, V2


#### Paso 3: Composición final de la SVD

Una vez tenemos los dos primeros pasos:

$$
A = U_1\, B\, V_1^T
$$ 

$$
B = U_2\, \Sigma\, V_2^T
$$

entonces la SVD completa de \(A\) es

$$
A = (U_1 U_2)\, \Sigma\, (V_1 V_2)^T.
$$




In [30]:
def svd_basic(A, tol=1e-12):
    '''Calcula la SVD de una matriz A usando reducción bidiagonal y SVD de bidiagonal.'''
    # Paso 1: reducción bidiagonal
    U1, B, V1 = bidiagonal_reduction(A)

    # Paso 2: SVD de bidiagonal
    U2, s, V2 = svd_bidiagonal(B)

    # Paso 3: combinación
    U = U1 @ U2
    V = V1 @ V2
    return U, s, V


Verificación comparando con Numpy SVD

In [31]:
np.random.seed(0)
A_test = np.random.randn(5, 3)

print("Matriz de prueba A:")
print(A_test)
print(f"\nDimensiones: {A_test.shape}")

U_my, S_my, V_my = svd_basic(A_test)
U_np, S_np, Vt_np = np.linalg.svd(A_test, full_matrices=False)

print("\nMy valores singulares:", S_my)
print("NumPy valores singulares:", S_np)

A_my  = U_my @ np.diag(S_my) @ V_my.T
A_np  = U_np @ np.diag(S_np) @ Vt_np

print("\nReconstrucción del error (my SVD):   ", np.linalg.norm(A_test - A_my))
print("Reconstrucción del error (NumPy SVD):", np.linalg.norm(A_test - A_np))


print("\nChequeo de ortogonalidad (U_my):", np.linalg.norm(U_my.T @ U_my - np.eye(U_my.shape[1])))
print("Chequeo de ortogonalidad (V_my):", np.linalg.norm(V_my.T @ V_my - np.eye(V_my.shape[1])))

print("\nError en valores singulares:", np.linalg.norm(np.sort(S_my)[::-1] - np.sort(S_np)[::-1]))

Matriz de prueba A:
[[ 1.7641  0.4002  0.9787]
 [ 2.2409  1.8676 -0.9773]
 [ 0.9501 -0.1514 -0.1032]
 [ 0.4106  0.144   1.4543]
 [ 0.761   0.1217  0.4439]]

Dimensiones: (5, 3)

My valores singulares: [3.5379 2.149  0.7688]
NumPy valores singulares: [3.5379 2.149  0.7688]

Reconstrucción del error (my SVD):    2.3592239273284576e-15
Reconstrucción del error (NumPy SVD): 1.0292978492192716e-15

Chequeo de ortogonalidad (U_my): 1.0624409406823177e-15
Chequeo de ortogonalidad (V_my): 1.075434317384651e-15

Error en valores singulares: 8.95090418262362e-16


### 2. Aproximación de Rango Bajo

La aproximación de rango $k$ de una matriz se obtiene truncando la SVD:

$$A_k = \sum_{i=1}^{k} \sigma_i u_i v_i^T$$

Esta es la mejor aproximación de rango $k$ en el sentido de norma de Frobenius.

In [32]:
def low_rank_approximation(A, k):
    """Calcula la aproximación de rango k usando SVD"""
    U, S, Vt = svd_basic(A)
    
    # Truncar a los primeros k componentes
    U_k = U[:, :k]
    S_k = S[:k]
    Vt_k = Vt[:k, :]
    
    return U_k @ np.diag(S_k) @ Vt_k


# Prueba
np.random.seed(42)
A = np.random.randn(10, 8)

print("Matriz original A:")
print(f"Forma: {A.shape}")
print(f"Rango aproximado: {np.linalg.matrix_rank(A)}")

# Calcular SVD
U, S, Vt = svd_basic(A)
print(f"\nValores singulares: {S}")

# Aproximaciones de diferentes rangos
for k in [1, 2, 3, 5]:
    A_k = low_rank_approximation(A, k)
    error = np.linalg.norm(A - A_k, 'fro')
    print(f"\nRango {k}: error de Frobenius = {error:.4f}")


Matriz original A:
Forma: (10, 8)
Rango aproximado: 8

Valores singulares: [5.1031 4.2116 3.681  2.4449 2.2908 1.6601 1.4381 0.5234]

Rango 1: error de Frobenius = 11.0491

Rango 2: error de Frobenius = 10.7998

Rango 3: error de Frobenius = 10.8735

Rango 5: error de Frobenius = 10.9470


### 3. Pseudoinversa de Moore-Penrose
 
 La pseudoinversa se calcula como:
 
 $$A^+ = V \Sigma^+ U^T$$
 
 donde $\Sigma^+$ es la matriz diagonal con elementos $\frac{1}{\sigma_i}$ para $\sigma_i > 0$


In [36]:
def pseudo_inverse_svd(U, S, V, tol=1e-12):
    """Calcula la pseudoinversa usando SVD: A+ = V * S+ * U^T"""
    # Construir S+
    S_inv = np.array([1/s if s > tol else 0 for s in S])
    S_plus = np.diag(S_inv)

    # Pseudoinversa
    A_plus = V @ S_plus @ U.T
    return A_plus

def pseudo_inverse(A, tol=1e-12):
    """Calcula la pseudoinversa de Moore-Penrose usando SVD"""
    U, S, V = svd_basic(A, tol=tol)
    return pseudo_inverse_svd(U, S, V, tol=tol)

### 4. Resolución de Problemas de Mínimos Cuadrados

Para resolver el problema de mínimos cuadrados:

$$\min_x \|Ax - b\|_2$$

la solución óptima se obtiene mediante la pseudoinversa:

$$x = A^+ b = V \Sigma^+ U^T b$$

Esta solución minimiza la norma euclidiana del residuo $r = Ax - b$.

In [37]:
A_ls = np.random.randn(6, 3)
b = np.random.randn(6)

A_plus = pseudo_inverse(A_ls)
x_ls = A_plus @ b

x_np, *_ = np.linalg.lstsq(A_ls, b, rcond=None)

print("My solution:  ", x_ls)
print("NumPy solution:", x_np)
print("Difference:", np.linalg.norm(x_ls - x_np))


My solution:   [ 0.114   0.6052 -0.5904]
NumPy solution: [ 0.114   0.6052 -0.5904]
Difference: 1.0565363201361521e-15


#### Propiedades de la solución:
- Si $A$ tiene rango completo por columnas, $x$ es único
- Si el sistema está sobredeterminado ($m > n$), $x$ es la solución de mínimos cuadrados
- Si el sistema está subdeterminado ($m < n$), $x$ es la solución de norma mínima

In [ ]:
# Verificación de propiedades de la pseudoinversa
A = np.random.randn(4, 3)
A_plus = pseudo_inverse(A)

# Propiedades que debe cumplir:
# 1. A @ A+ @ A = A
# 2. A+ @ A @ A+ = A+
# 3. (A @ A+)^T = A @ A+
# 4. (A+ @ A)^T = A+ @ A

print("Propiedades de la pseudoinversa:")
print(f"1. ||A @ A+ @ A - A|| = {np.linalg.norm(A @ A_plus @ A - A):.2e}")
print(f"2. ||A+ @ A @ A+ - A+|| = {np.linalg.norm(A_plus @ A @ A_plus - A_plus):.2e}")
print(f"3. ||(A @ A+)^T - A @ A+|| = {np.linalg.norm((A @ A_plus).T - A @ A_plus):.2e}")
print(f"4. ||(A+ @ A)^T - A+ @ A|| = {np.linalg.norm((A_plus @ A).T - A_plus @ A):.2e}")

Propiedades de la pseudoinversa:
1. ||A @ A+ @ A - A|| = 1.20e-15
2. ||A+ @ A @ A+ - A+|| = 1.83e-15
3. ||(A @ A+)^T - A @ A+|| = 1.60e-15
4. ||(A+ @ A)^T - A+ @ A|| = 1.23e-15
